In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_curve, auc, precision_score, recall_score
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import LocalOutlierFactor
from numpy import arange
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report

In [2]:
benign_df = pd.read_csv(r'D:\Kuliah\ProyekAkhir\New Data\resample\Benign_resample.csv')
benign_df.head(2)

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,15.854422,-0.343690,-0.086569,0.096351,0.142000,0.002723,0.706619,1.982467,1.798333,-0.034669,...,-0.059807,-0.095769,-0.109896,-0.112787,-0.020891,-0.339324,-0.282883,-0.341153,-0.315336,1
1,-0.060352,5.598377,2.253048,4.681794,3.829148,0.125465,3.569438,-0.039001,2.481753,5.125594,...,-0.059807,1.103561,0.162506,0.515708,1.673959,6.184780,12.942534,8.511326,2.479016,1


In [3]:
udplag_df = pd.read_csv(r'D:\Kuliah\ProyekAkhir\New Data\resample\UDPLag_resample_4000.csv')
udplag_df

,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,2.209517,-0.327036,-0.309612,-0.339661,0.650963,-0.010439,2.084232,2.355482,2.257843,-0.198358,...,0.063244,-0.098072,-0.10727,-0.109441,-0.015768,-0.315335,-0.268473,-0.315664,-0.295924,-1
1,2.209517,-0.327036,-0.309612,-0.339661,0.650963,-0.010439,2.084232,2.355482,2.257843,-0.198358,...,0.063244,-0.098072,-0.10727,-0.109441,-0.015768,-0.315335,-0.268473,-0.315664,-0.295924,-1
2,2.209517,-0.327036,-0.309612,-0.339661,0.656191,-0.010439,2.097862,2.370592,2.272441,-0.198358,...,0.063244,-0.098072,-0.10727,-0.109441,-0.015768,-0.315335,-0.268473,-0.315664,-0.295924,-1
3,2.209517,-0.327036,-0.309612,-0.339661,0.656191,-0.010439,2.097862,2.370592,2.272441,-0.198358,...,0.063244,-0.098072,-0.10727,-0.109441,-0.015768,-0.315335,-0.268473,-0.315664,-0.295924,-1
4,2.209517,-0.327036,-0.309612,-0.339661,0.624822,-0.010439,2.016083,2.279932,2.184854,-0.198358,...,0.063244,-0.098072,-0.10727,-0.109441,-0.015768,-0.315335,-0.268473,-0.315664,-0.295924,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1868,2.209517,-0.327036,-0.309612,-0.339661,0.820881,-0.010439,2.527200,2.846555,2.732269,-0.198358,...,0.063244,-0.098072,-0.10727,-0.109441,-0.015768,-0.315335,-0.268473,-0.315664,-0.295924,-1
1869,2.209517,-0.327036,-0.309612,-0.339661,0.820881,-0.010439,2.527200,2.846555,2.732269,-0.198358,...,0.063244,-0.098072,-0.10727,-0.109441,-0.015768,-0.315335,-0.268473,-0.315664,-0.295924,-1
1870,2.209517,-0.327036,-0.309612,-0.339661,0.820881,-0.010439,2.527200,2.846555,2.732269,-0.198358,...,0.063244,-0.098072,-0.10727,-0.109441,-0.015768,-0.315335,-0.268473,-0.315664,-0.295924,-1
1871,2.209517,-0.327036,-0.309612,-0.339661,0.820881,-0.010439,2.527200,2.846555,2.732269,-0.198358,...,0.063244,-0.098072,-0.10727,-0.109441,-0.015768,-0.315335,-0.268473,-0.315664,-0.295924,-1


In [4]:
benign_test_df = pd.read_csv(r'D:\Kuliah\ProyekAkhir\New Data\resample\Benign_resample_4000.csv')

In [5]:
# novelty detection
df_test = pd.concat([benign_test_df, udplag_df], ignore_index=True)
df_train = benign_df

In [6]:
df_test = df_test[[' Flow Duration', ' Fwd Packet Length Min', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', ' Flow Packets/s', 'Fwd IAT Total', 'Fwd PSH Flags', 'Fwd Packets/s', ' Min Packet Length', ' Packet Length Std', ' RST Flag Count', ' ACK Flag Count', ' URG Flag Count', ' CWE Flag Count',  ' Down/Up Ratio', ' Avg Bwd Segment Size', ' min_seg_size_forward',' Label']]
df_train = df_train[[' Flow Duration', ' Fwd Packet Length Min', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', ' Flow Packets/s', 'Fwd IAT Total', 'Fwd PSH Flags', 'Fwd Packets/s', ' Min Packet Length', ' Packet Length Std', ' RST Flag Count', ' ACK Flag Count', ' URG Flag Count', ' CWE Flag Count',  ' Down/Up Ratio', ' Avg Bwd Segment Size', ' min_seg_size_forward',' Label']]


In [7]:
# training_data
X_train = df_train.drop([' Label'], axis=1)
y_train = df_train[' Label']
# testing_data
X_test = df_test.drop([' Label'], axis=1)
y_test = df_test[' Label']

In [9]:
# Novelty detection with 8000 sample balanced
for a in range(1, 5):
    for i in arange(0.1, 0.5, 0.1):
        clf = LocalOutlierFactor(n_neighbors=a, novelty=True, contamination=i)
        clf.fit(X_train)
        y_pred_test = clf.predict(X_test)
        print('Neighbour : ' + str(a))
        print('contamination : ' + str(i))
        print('---Classification Report---')
        print(classification_report(y_test, y_pred_test))

Neighbour : 1
contamination : 0.1
---Classification Report---
              precision    recall  f1-score   support

          -1       1.00      0.99      1.00      1873
           1       1.00      1.00      1.00      4003

    accuracy                           1.00      5876
   macro avg       1.00      1.00      1.00      5876
weighted avg       1.00      1.00      1.00      5876

Neighbour : 1
contamination : 0.2
---Classification Report---
              precision    recall  f1-score   support

          -1       1.00      0.99      1.00      1873
           1       1.00      1.00      1.00      4003

    accuracy                           1.00      5876
   macro avg       1.00      1.00      1.00      5876
weighted avg       1.00      1.00      1.00      5876

Neighbour : 1
contamination : 0.30000000000000004
---Classification Report---
              precision    recall  f1-score   support

          -1       1.00      0.99      1.00      1873
           1       1.00      1.00  